In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import sys

In [5]:
# !pip download scikit-maad==1.4.1 -d /kaggle/working/mysitepackages/sklearn_maad_pkg
#!pip download ffmpeg-python==0.2.0 -d /kaggle/working/mysitepackages/ffmpeg_pkg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 67.8 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 69.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 73.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 37.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 85.5 MB/s eta

In [29]:
"""from zipfile import ZipFile
import os

dirName = '/kaggle/working/mysitepackages'
zipName = 'ffmpeg_pkg.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)"""

In [1]:
#!pip install ffmpeg-python --no-index --find-links=file:///kaggle/input/ffmpeg-pkg/kaggle/working
#!pip install scikit-maad --no-index --find-links=file:///kaggle/input/sklearn-maad-offline/kaggle/working/mysitepackages/sklearn_maad_pkg

Looking in links: file:///kaggle/input/ffmpeg-pkg/kaggle/working
Processing /kaggle/input/ffmpeg-pkg/kaggle/working/ffmpeg_python-0.2.0-py3-none-any.whl


In [3]:
"""from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from maad import sound, rois, features
from maad.util import (
   power2dB, plot2d, format_features, read_audacity_annot,
   overlay_rois, overlay_centroid
   )"""

In [2]:
# Function to extract features from audio file
"""def extract_features(file_path, offset=0.0, duration=5.0):
    # Load audio file
    audio, sample_rate = librosa.load(path=file_path, sr=32000, offset=offset, duration=duration)
    # Extract features using Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # Flatten the features into a 1D array
    flattened_features = np.mean(mfccs.T, axis=0)
    return flattened_features

def audio_classification(file_path, offset=0.0, duration=5.0):
    extracted_features = extract_features(file_path, offset=offset, duration=duration).reshape(1, -1)
    y_predict = random_forest_model.predict_proba(extracted_features)
    
    return y_predict"""

In [4]:
test_soundscapes = '/kaggle/input/birdclef-2024/unlabeled_soundscapes'
# test_soundscapes = '/kaggle/input/birdclef-2024/test_soundscapes'

filenames_with_path = glob.glob(f"{test_soundscapes}/*.ogg")
# filenames = [('soundscape_' + os.path.basename(filename)) for filename in filenames_with_path]
filenames = [os.path.basename(filename) for filename in filenames_with_path]

print(len(filenames))

8444


In [5]:
test_files = filenames[7000:7010]
len(test_files)

10

In [8]:
# root_path = '/kaggle/input/birdclef-2024/test_soundscapes/'
root_path = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/'

durations = {}

for file in tqdm(test_files):
    full_path = root_path+file
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    durations[file] = round(len(audio) / sample_rate)

100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


In [10]:
chunk_dict = {}

for file in tqdm(test_files):
    full_path = root_path+file
    
    if durations[file] <= 240:
        duration = durations[file]+5
    else:
        duration = 240+5
        
    for j in range(5,duration,5):
        f_offset = float(j-5)
        audio, sample_rate = librosa.load(path=full_path, sr=32000, offset=f_offset, duration=5.0)
        cur_file = file.replace('.ogg','')
        chunk = cur_file+'_'+str(j)
        mfccs = librosa.feature.mfcc(y=audio, sr=32000, n_mfcc=40)
        # Flatten the features into a 1D array
        flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
        chunk_dict[chunk] = flattened_features

100%|██████████| 10/10 [00:23<00:00,  2.38s/it]


In [12]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)
bird_cols = list(meta_data['primary_label'].unique().sort())

submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit['row_id'] = 'samples'

submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
1,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
2,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495


In [13]:
random_forest_model = load('/kaggle/input/trained-audio-model-v-01/audio_classifier_model.joblib')

In [14]:
# this is where the inference loop will go

for file_name, feature_set in tqdm(chunk_dict.items()):
    df = pd.DataFrame(random_forest_model.predict_proba(feature_set), columns=bird_cols)
    df.insert(loc=0, column='row_id', value=file_name)
    submit = pd.concat([submit,df]).reset_index(drop=True)
    
i = submit[(submit.row_id == 'samples')].index
submit = submit.drop(i).reset_index(drop=True)

100%|██████████| 480/480 [00:04<00:00, 97.17it/s] 


In [ ]:
"""for i in range(len(filenames_with_path)):
    audio, sample_rate = librosa.load(path=filenames_with_path[i], sr=32000)
    duration = round(len(audio) / sample_rate)
    
    # now = datetime.now()
    for j in range(5,duration,5):
        offset = j-5
        predicted = audio_classification(filenames_with_path[i], offset=float(offset), duration=5.0)
        filename_noext = filenames[i].replace('.ogg','')
        # print(f'{filename_noext}_{j}')
        submit['row_id'] = filename_noext+'_'+str(j)
        submit[bird_cols] = predicted.round(5)
        # print(submit['row_id'])
        submission_df = pd.concat([submission_df,submit]).reset_index(drop=True)"""

In [27]:
#Audio(audio, rate=32000)

In [15]:
#submit.sum(axis=1, numeric_only=True)

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
475    1.0
476    1.0
477    1.0
478    1.0
479    1.0
Length: 480, dtype: float64

In [16]:
submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,545884992_5,0.02,0.01,0.01,0.01,0.00,0.00,0.01,0.00,0.0,...,0.02,0.00,0.00,0.01,0.00,0.02,0.0,0.00,0.00,0.00
1,545884992_10,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.02,0.01,0.00,0.00,0.01,0.01,0.0,0.00,0.01,0.01
2,545884992_15,0.00,0.00,0.01,0.02,0.00,0.01,0.01,0.00,0.0,...,0.03,0.01,0.00,0.00,0.03,0.00,0.0,0.00,0.00,0.01
3,545884992_20,0.00,0.01,0.00,0.02,0.01,0.00,0.00,0.00,0.0,...,0.02,0.01,0.00,0.00,0.01,0.00,0.0,0.00,0.01,0.00
4,545884992_25,0.00,0.01,0.00,0.00,0.03,0.00,0.00,0.00,0.0,...,0.01,0.01,0.00,0.00,0.02,0.00,0.0,0.00,0.01,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,1983759531_220,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.0,...,0.02,0.00,0.01,0.00,0.03,0.02,0.0,0.00,0.00,0.00
476,1983759531_225,0.00,0.00,0.02,0.00,0.01,0.01,0.00,0.00,0.0,...,0.00,0.00,0.01,0.01,0.03,0.01,0.0,0.03,0.00,0.00
477,1983759531_230,0.01,0.00,0.02,0.00,0.02,0.00,0.01,0.00,0.0,...,0.03,0.01,0.02,0.00,0.03,0.02,0.0,0.00,0.01,0.00
478,1983759531_235,0.01,0.00,0.02,0.00,0.01,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.01,0.01,0.01,0.0,0.02,0.00,0.00


In [17]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submit.columns)
#submission_df = submission_df[sample_submission.columns]

In [18]:
submit.to_csv('submission.csv', index=False)